In [132]:
from pycorese.api import CoreseAPI

python_to_java_bridge = 'py4j'
corese = CoreseAPI(java_bridge=python_to_java_bridge)
_ = corese.loadCorese()


import os

if 'src' in os.getcwd():
    os.chdir('..')

os.getcwd()

2025-04-19 12:21:05,698 - INFO - Py4J: Loading CORESE...
2025-04-19 12:21:07,637 - INFO - Py4J: CORESE is loaded


'/mnt/c/repos/rdf_permissions_example'

In [133]:
g = corese.loadRDF("data/server/graph_data.rdf")

In [134]:
# Simple first 5 triples from the whole graph
res = corese.sparqlSelect(g, "SELECT * WHERE { ?s ?p ?o } LIMIT 5", return_dataframe=True)
res

,s,p,o
0,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#managedBy
1,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#manages...
2,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,_:b1
3,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,http://www.w3.org/1999/02/22-rdf-syntax-ns#nil
4,local://security.permissions.net/entities#1787...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,local://security.permissions.net/vocab#Record


In [135]:
# Since we haven't run ontological reasoning yet, the "manages" relationships are not inferred
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#APPAREL
1,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
2,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE
3,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN
4,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#CHRI...
5,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#BIRT...
6,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...
7,local://security.permissions.net/entities#Anya,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#STORAGE


In [136]:
# Run the rules engine
corese.resetRuleEngine(g)
corese.loadRuleEngine(g, profile='owlrl')
corese.loadRuleEngine(g, profile='owlrl_lite')
corese.loadRuleEngine(g, profile='owlrl_ext')
corese.loadRuleEngine(g, profile='rdfs')

JavaObject id=o18

In [137]:
# Now that we have added our inferences, we can see the results of the manages relationships which have been inferred from both managesDepartment and mangedBy
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
1,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#APPAREL
2,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
3,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
4,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
...,...,...,...
61,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN
62,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#CHRI...
63,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#BIRT...
64,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...


In [138]:
# Find all the records that a specific user may have access to.

USER = "SalesRep_0"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select distinct ?record where {{
data:{USER} :hasAccess ?item . 
?item a :Item .
?item :hasRecord ?record .
?record a :Record .
optional {{
    data:{USER} :hasAccess ?country .
    ?country a :Country ; :hasRecord ?record .
}}
optional {{
    data:{USER} a :Manager .
    bind(:Manager as ?mgr) .
}}
filter(bound(?country) || bound(?mgr)) .
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record
0,local://security.permissions.net/entities#4566...
1,local://security.permissions.net/entities#4566...
2,local://security.permissions.net/entities#4566...
3,local://security.permissions.net/entities#4566...
4,local://security.permissions.net/entities#4566...
...,...
2105,local://security.permissions.net/entities#4457...
2106,local://security.permissions.net/entities#4654...
2107,local://security.permissions.net/entities#1785...
2108,local://security.permissions.net/entities#1785...


In [ ]:
# Find all the records that a specific user may have access to. This query is simpler to read and more performant

USER = "SalesRep_0"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select distinct ?record where {{
    {{
    data:{USER} a :SalesRep; :hasAccess ?country, ?item .
    ?country a :Country ; :hasRecord ?record .
    ?item a :Item ; :hasRecord ?record .
    }} union {{
    data:{USER} a :Manager; :hasAccess ?item .
    ?item a :Item ; :hasRecord ?record .
    }}
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record
0,local://security.permissions.net/entities#4566...
1,local://security.permissions.net/entities#1787...
2,local://security.permissions.net/entities#1779...
3,local://security.permissions.net/entities#4559...
4,local://security.permissions.net/entities#4667...
...,...
2105,local://security.permissions.net/entities#4823...
2106,local://security.permissions.net/entities#4671...
2107,local://security.permissions.net/entities#4488...
2108,local://security.permissions.net/entities#4672...


In [ ]:
# Check the records for a manager as well

USER = "Mary"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select distinct ?record where {{
    {{
    data:{USER} a :SalesRep; :hasAccess ?country, ?item .
    ?country a :Country ; :hasRecord ?record .
    ?item a :Item ; :hasRecord ?record .
    }} union {{
    data:{USER} a :Manager; :hasAccess ?item .
    ?item a :Item ; :hasRecord ?record .
    }}
}}
"""

# Execute the query
res2 = corese.sparqlSelect(g, query, return_dataframe=True)
res2

,record
0,local://security.permissions.net/entities#4654...
1,local://security.permissions.net/entities#4745...
2,local://security.permissions.net/entities#4745...
3,local://security.permissions.net/entities#4745...
4,local://security.permissions.net/entities#4773...
...,...
1164,local://security.permissions.net/entities#4908...
1165,local://security.permissions.net/entities#1785...
1166,local://security.permissions.net/entities#4582...
1167,local://security.permissions.net/entities#4582...


# Showcase handling of the canSellSet Permission

## Wihtout Applying

In [169]:
# Find all the records that a specific user may have access to. This query is simpler to read and more performant

USER = "SalesRep_2"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select distinct ?record ?is_set ?sell_sets_permission where {{
    {{
    data:{USER} a :SalesRep; :hasAccess ?country, ?item ; :canSellSets ?sell_sets_permission .
    ?country a :Country ; :hasRecord ?record .
    ?item a :Item ; :hasRecord ?record ; :isSet ?is_set .
    }} union {{
    data:{USER} a :Manager; :hasAccess ?item .
    ?item a :Item ; :hasRecord ?record .
    }}
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record,is_set,sell_sets_permission
0,local://security.permissions.net/entities#4649...,True,False
1,local://security.permissions.net/entities#4891...,True,False
2,local://security.permissions.net/entities#4708...,False,False
3,local://security.permissions.net/entities#4643...,False,False
4,local://security.permissions.net/entities#4881...,True,False
...,...,...,...
61,local://security.permissions.net/entities#7520...,False,False
62,local://security.permissions.net/entities#4679...,False,False
63,local://security.permissions.net/entities#4695...,False,False
64,local://security.permissions.net/entities#4812...,False,False


## Apply with `minus` operation

In [170]:
# Find all the records that a specific user may have access to. This query is simpler to read and more performant

USER = "SalesRep_2"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select distinct ?record ?is_set ?sell_sets_permission where {{
    {{
    data:{USER} a :SalesRep; :hasAccess ?country, ?item ; :canSellSets ?sell_sets_permission .
    ?country a :Country ; :hasRecord ?record .
    ?item a :Item ; :hasRecord ?record ; :isSet ?is_set .
    }} union {{
    data:{USER} a :Manager; :hasAccess ?item .
    ?item a :Item ; :hasRecord ?record .
    }}
    minus {{
        data:{USER} :hasAccess ?item ; :canSellSets false .
        ?item :isSet true .
    }}
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record,is_set,sell_sets_permission
0,local://security.permissions.net/entities#4708...,False,False
1,local://security.permissions.net/entities#4643...,False,False
2,local://security.permissions.net/entities#4820...,False,False
3,local://security.permissions.net/entities#4511...,False,False
4,local://security.permissions.net/entities#4643...,False,False
5,local://security.permissions.net/entities#1765...,False,False
6,local://security.permissions.net/entities#4614...,False,False
7,local://security.permissions.net/entities#4866...,False,False
8,local://security.permissions.net/entities#4468...,False,False
9,local://security.permissions.net/entities#4751...,False,False


## Show that this update would not affect the managers

In [171]:
# Find all the records that a specific user may have access to. This query is simpler to read and more performant

USER = "Mary"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select distinct ?record ?is_set ?sell_sets_permission where {{
    {{
    data:{USER} a :SalesRep; :hasAccess ?country, ?item ; :canSellSets ?sell_sets_permission .
    ?country a :Country ; :hasRecord ?record .
    ?item a :Item ; :hasRecord ?record ; :isSet ?is_set .
    }} union {{
    data:{USER} a :Manager; :hasAccess ?item .
    ?item a :Item ; :hasRecord ?record .
    }}
    minus {{
        data:{USER} :hasAccess ?item ; :canSellSets false .
        ?item :isSet true .
    }}
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record,is_set,sell_sets_permission
0,local://security.permissions.net/entities#4654...,<NA>,<NA>
1,local://security.permissions.net/entities#4745...,<NA>,<NA>
2,local://security.permissions.net/entities#4745...,<NA>,<NA>
3,local://security.permissions.net/entities#4745...,<NA>,<NA>
4,local://security.permissions.net/entities#4773...,<NA>,<NA>
...,...,...,...
1164,local://security.permissions.net/entities#4908...,<NA>,<NA>
1165,local://security.permissions.net/entities#1785...,<NA>,<NA>
1166,local://security.permissions.net/entities#4582...,<NA>,<NA>
1167,local://security.permissions.net/entities#4582...,<NA>,<NA>
